In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import DataUtils
import Masking

C:\Users\austi\miniconda3\envs\deep_learn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 64

In [3]:
columns_excluded = [0, 1, 26]
columns_kept = [False, False, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, True, True, True, True,
               True, False]
print(columns_kept)

[False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False]


In [4]:
trainloader, testloader = DataUtils.get_dataloaders(batch_size)

In [5]:
example = next(enumerate(trainloader))

In [6]:
ex_X = example[1][0]
print(ex_X.shape)

ex_y = example[1][1]
print(ex_y.shape)

torch.Size([64, 543, 27])
torch.Size([64, 543])


In [7]:
# Should we use a linear attention transformer?
class ImputationTransformer(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        
        self.input_projection = nn.Linear(543*24, embed_dim)
        self.positional_embed = nn.Parameter(torch.randn(embed_dim))
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=4, activation="gelu")
        self.transformer_blocks = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
    
    def forward(self, x):
        x = x.flatten(1)
        z = self.input_projection(x)
        z = z + self.positional_embed
        z = self.transformer_blocks(z)
        
        return z

In [8]:
class ReconstructionImputationTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed_dim = 128
        self.imputation_transformer = ImputationTransformer(self.embed_dim)
        self.reconstruction = nn.Sequential(nn.Linear(self.embed_dim, 2048),
                                           nn.ReLU(),
                                           nn.Linear(2048, 543*24))
    
    def forward(self, x):
        z = self.imputation_transformer(x)
        z = self.reconstruction(z)
        
        return z.view(x.shape[0],543,24)

In [9]:
ex_X[:,:,columns_kept].shape

torch.Size([64, 543, 24])

In [10]:
r_tran = ReconstructionImputationTransformer()
print("Num params:",sum(p.numel() for p in r_tran.parameters() if p.requires_grad))

Num params: 31007208


In [11]:
r_tran(ex_X[:,:,columns_kept].float()).shape

torch.Size([64, 543, 24])

### Training Model

In [ ]:
def mse_missing(xhat, masked_X, X):
    missing_idx = (masked_X==float(-1))
    error = xhat[missing_idx] - X[missing_idx]
    return torch.mean(torch.pow(error, 2))

In [13]:
objective = mse_missing

r_tran = ReconstructionImputationTransformer()

lr = 1e-4
n_epochs = 5
optim = torch.optim.Adam(r_tran.parameters(), lr=lr)
losses = []

for n in range(n_epochs):
    counter = 0
    for i, (X, y) in enumerate(tqdm(trainloader)):
        
        masked_X = Masking.mask_input(X)
        optim.zero_grad()
        xhat = r_tran(masked_X[:,:,columns_kept].float())
        loss = objective(xhat, masked_X[:,:,columns_kept], X[:,:,columns_kept])\
        loss.backward()
        losses.append(loss.item())
        optim.step()
        counter += 1
        
    print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

tensor([-0.6659, -0.4141,  0.0926,  ..., -1.0164, -0.0238, -0.4041],
       dtype=torch.float64, grad_fn=<SubBackward0>)
tensor(0.4277, dtype=torch.float64, grad_fn=<MeanBackward0>)


  8%|██████▉                                                                            | 1/12 [00:03<00:40,  3.70s/it]

tensor([-0.8668, -0.9383, -0.9288,  ..., -0.0895, -0.1473, -0.1360],
       dtype=torch.float64, grad_fn=<SubBackward0>)
tensor(0.3640, dtype=torch.float64, grad_fn=<MeanBackward0>)


 17%|█████████████▊                                                                     | 2/12 [00:07<00:36,  3.69s/it]

...

tensor([ 0.2701,  0.3177, -0.0201,  ...,  0.7157,  0.1877,  0.2943],
       dtype=torch.float64, grad_fn=<SubBackward0>)
tensor(0.1118, dtype=torch.float64, grad_fn=<MeanBackward0>)


 92%|███████████████████████████████████████████████████████████████████████████▏      | 11/12 [00:40<00:03,  3.65s/it]

tensor([-0.2568, -0.1330,  0.2080,  ..., -0.0413,  0.3477,  0.5621],
       dtype=torch.float64, grad_fn=<SubBackward0>)
tensor(0.0889, dtype=torch.float64, grad_fn=<MeanBackward0>)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:40<00:00,  3.39s/it]

Epoch: 5 Loss: 0.09182464163939537


In [14]:
with torch.no_grad():
    print(mse_missing(xhat, masked_X[:,:,columns_kept], X[:,:,columns_kept]))

tensor([-0.2568, -0.1330,  0.2080,  ..., -0.0413,  0.3477,  0.5621],
       dtype=torch.float64)
tensor(0.0889, dtype=torch.float64)


In [15]:
test_mses = []
with torch.no_grad():
    for i, (X, y) in enumerate(tqdm(testloader)):
            masked_X = Masking.mask_input(X)
            xhat = r_tran(masked_X[:,:,columns_kept].float())
            test_mse = mse_missing(xhat, masked_X[:,:,columns_kept], X[:,:,columns_kept]).item()
            test_mses.append(test_mse.item())
print("Test MSE:", np.mean(test_mses))

  0%|                                                                                           | 0/12 [00:01<?, ?it/s]

tensor([ 0.0204, -0.0875, -0.2634,  ..., -0.1168, -0.0523, -0.0008],
       dtype=torch.float64)


TypeError: mse_missing() missing 1 required positional argument: 'X'